# **Chapter 24: Native Android Integration**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Embed Flutter within existing Android Activities and Fragments for incremental adoption
- Implement bidirectional communication between Kotlin/Java and Dart using platform channels
- Configure AndroidManifest.xml for permissions, intent filters, and app links
- Handle incoming intents and deep links from other applications
- Implement background processing using WorkManager and foreground services
- Manage Android-specific lifecycle events and resource cleanup

---

## **Prerequisites**

- Completed Chapter 23: Platform Integration (MethodChannel, PlatformView basics)
- Intermediate knowledge of Android development (Activity/Fragment lifecycle, Intents)
- Understanding of Kotlin coroutines and Android threading model
- Familiarity with Android Manifest configuration and Gradle build system
- Access to Android Studio for native code editing

---

## **24.1 Android Activity & Fragment Embedding**

Flutter can be embedded into existing Android applications as a Fragment or Activity, enabling incremental migration of native apps to Flutter.

### **FlutterActivity Configuration**

```kotlin
// android/app/src/main/kotlin/com/example/app/MainActivity.kt
// Standard FlutterActivity - Entry point for full Flutter app

package com.example.app

import io.flutter.embedding.android.FlutterActivity
import io.flutter.embedding.engine.FlutterEngine
import io.flutter.plugin.common.MethodChannel

class MainActivity : FlutterActivity() {
    
    // Override to configure FlutterEngine before launch
    override fun configureFlutterEngine(flutterEngine: FlutterEngine) {
        super.configureFlutterEngine(flutterEngine)
        
        // Register plugins and platform channels
        setupMethodChannels(flutterEngine)
    }
    
    private fun setupMethodChannels(engine: FlutterEngine) {
        MethodChannel(engine.dartExecutor.binaryMessenger, "app_channel")
            .setMethodCallHandler { call, result ->
                when (call.method) {
                    "getAndroidVersion" -> {
                        result.success(android.os.Build.VERSION.RELEASE)
                    }
                    "finishActivity" -> {
                        // Programmatically close activity from Flutter
                        finish()
                        result.success(true)
                    }
                    else -> result.notImplemented()
                }
            }
    }
    
    // Handle activity results from native intents
    override fun onActivityResult(requestCode: Int, resultCode: Int, data: Intent?) {
        super.onActivityResult(requestCode, resultCode, data)
        
        // Forward to Flutter if needed, or handle natively
        if (requestCode == REQUEST_CODE_IMAGE_PICK && resultCode == RESULT_OK) {
            data?.data?.let { uri ->
                // Process selected image URI
                val path = getRealPathFromURI(uri)
                // Send to Flutter via platform channel
            }
        }
    }
    
    // Handle back button press
    override fun onBackPressed() {
        // Intercept back press, can delegate to Flutter
        // Or handle natively for hybrid flows
        super.onBackPressed()
    }
    
    companion object {
        const val REQUEST_CODE_IMAGE_PICK = 1001
    }
}
```

**Explanation:**

- **FlutterActivity**: The base class that hosts the Flutter engine. It manages the FlutterView lifecycle, orientation changes, and configuration updates.
- **configureFlutterEngine**: Called once when the activity is created. This is where you register MethodChannels before Flutter starts rendering. This ensures channels are available immediately when Dart code runs.
- **onActivityResult**: Android's callback for results from other activities (image picker, camera, etc.). The requestCode identifies which operation returned.
- **getRealPathFromURI**: Android content URIs (`content://`) must be resolved to file paths. This requires querying the MediaStore content resolver (implementation omitted for brevity).

### **FlutterFragment Embedding**

```kotlin
// android/app/src/main/kotlin/com/example/app/NativeHostActivity.kt
// Hosting Flutter as a Fragment within native Android UI

package com.example.app

import android.os.Bundle
import android.widget.Button
import android.widget.FrameLayout
import androidx.appcompat.app.AppCompatActivity
import io.flutter.embedding.android.FlutterFragment
import io.flutter.embedding.android.RenderMode
import io.flutter.embedding.android.TransparencyMode
import io.flutter.embedding.engine.FlutterEngine
import io.flutter.embedding.engine.FlutterEngineCache
import io.flutter.embedding.engine.dart.DartExecutor

class NativeHostActivity : AppCompatActivity() {
    
    // Tag for fragment transaction
    private val flutterFragmentTag = "flutter_fragment"
    private var flutterFragment: FlutterFragment? = null
    
    // Cached engine for faster startup
    companion object {
        const val ENGINE_ID = "my_engine_id"
    }
    
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_native_host)
        
        // Initialize Flutter engine early (optional optimization)
        initFlutterEngine()
        
        // Setup native UI
        findViewById<Button>(R.id.showFlutterButton).setOnClickListener {
            showFlutterFragment()
        }
        
        findViewById<Button>(R.id.hideFlutterButton).setOnClickListener {
            hideFlutterFragment()
        }
        
        // Or embed Flutter immediately
        if (savedInstanceState == null) {
            createFlutterFragment()
        }
    }
    
    private fun initFlutterEngine() {
        // Cache engine for warm start
        if (FlutterEngineCache.getInstance().get(ENGINE_ID) == null) {
            val flutterEngine = FlutterEngine(this)
            
            // Pre-warm engine by starting Dart execution
            flutterEngine.dartExecutor.executeDartEntrypoint(
                DartExecutor.DartEntrypoint.createDefault()
            )
            
            // Cache engine for reuse
            FlutterEngineCache.getInstance().put(ENGINE_ID, flutterEngine)
        }
    }
    
    private fun createFlutterFragment() {
        // Configure FlutterFragment
        flutterFragment = FlutterFragment.withCachedEngine(ENGINE_ID)
            .renderMode(RenderMode.texture)  // texture or surface
            .transparencyMode(TransparencyMode.transparent)
            .shouldAttachEngineToActivity(true)  // For platform channel access
            .build()
        
        // Add to container
        supportFragmentManager
            .beginTransaction()
            .add(R.id.flutter_container, flutterFragment!!, flutterFragmentTag)
            .commit()
    }
    
    private fun showFlutterFragment() {
        flutterFragment?.let { fragment ->
            supportFragmentManager
                .beginTransaction()
                .show(fragment)
                .commit()
        }
    }
    
    private fun hideFlutterFragment() {
        flutterFragment?.let { fragment ->
            supportFragmentManager
                .beginTransaction()
                .hide(fragment)
                .commit()
        }
    }
    
    // Handle back button for Fragment
    override fun onBackPressed() {
        // Delegate to FlutterFragment first
        flutterFragment?.let {
            if (it.onBackPressed()) {
                return  // Flutter handled it
            }
        }
        super.onBackPressed()
    }
    
    override fun onDestroy() {
        // Only destroy engine if not caching
        // FlutterEngineCache.getInstance().remove(ENGINE_ID)
        super.onDestroy()
    }
}
```

**Explanation:**

- **FlutterFragment**: Embeds Flutter in a native Android layout alongside native widgets. Use this for hybrid screens (e.g., native app bar with Flutter content).
- **RenderMode.texture**: Renders Flutter to an Android TextureView, supports transparency and overlapping UI but lower performance than surface mode.
- **RenderMode.surface**: Uses SurfaceView, better performance but cannot be transparent and has synchronization issues with platform views.
- **Engine Caching**: Pre-warming and caching the FlutterEngine in `Application.onCreate()` or activity `onCreate()` reduces cold start time from 2-3 seconds to <500ms.
- **shouldAttachEngineToActivity**: When true, the FlutterEngine receives activity lifecycle events and can access Activity-based platform channels.

### **Activity Layout XML**

```xml
<!-- android/app/src/main/res/layout/activity_native_host.xml -->
<LinearLayout 
    xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical">

    <!-- Native Android UI -->
    <androidx.appcompat.widget.Toolbar
        android:id="@+id/toolbar"
        android:layout_width="match_parent"
        android:layout_height="?attr/actionBarSize"
        android:background="?attr/colorPrimary" />

    <Button
        android:id="@+id/showFlutterButton"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Show Flutter" />

    <!-- Flutter Fragment Container -->
    <FrameLayout
        android:id="@+id/flutter_container"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1" />

</LinearLayout>
```

---

## **24.2 Kotlin/Java Interop with Flutter**

Advanced platform channel patterns including async operations, callbacks, and complex data structures.

### **Async Platform Channel Implementation**

```kotlin
// android/app/src/main/kotlin/com/example/app/channels/AsyncMethodChannel.kt

package com.example.app.channels

import io.flutter.plugin.common.MethodCall
import io.flutter.plugin.common.MethodChannel
import kotlinx.coroutines.*
import kotlin.coroutines.CoroutineContext

class AsyncMethodChannel(
    private val channel: MethodChannel,
    private val mainScope: CoroutineScope = MainScope()
) : CoroutineScope by mainScope {
    
    init {
        channel.setMethodCallHandler { call, result ->
            // Launch coroutine for async handling
            launch {
                handleMethodCall(call, result)
            }
        }
    }
    
    private suspend fun handleMethodCall(call: MethodCall, result: MethodChannel.Result) {
        when (call.method) {
            "fetchUserData" -> {
                try {
                    val userId = call.argument<String>("userId") 
                        ?: throw IllegalArgumentException("userId required")
                    
                    // Suspend function for network/DB operation
                    val userData = withContext(Dispatchers.IO) {
                        fetchUserFromNetwork(userId)
                    }
                    
                    // Result.success must be called on main thread
                    result.success(userData.toMap())
                } catch (e: Exception) {
                    result.error("FETCH_ERROR", e.message, e.stackTraceToString())
                }
            }
            
            "processImage" -> {
                val imagePath = call.argument<String>("path")
                val quality = call.argument<Int>("quality") ?: 80
                
                // Long-running operation with progress callbacks
                processImageAsync(imagePath, quality, result)
            }
            
            else -> result.notImplemented()
        }
    }
    
    private suspend fun fetchUserFromNetwork(userId: String): User {
        // Simulate network delay
        delay(1000)
        return User(
            id = userId,
            name = "John Doe",
            email = "john@example.com"
        )
    }
    
    private fun processImageAsync(
        imagePath: String?, 
        quality: Int,
        result: MethodChannel.Result
    ) {
        if (imagePath == null) {
            result.error("INVALID_PATH", "Image path is null", null)
            return
        }
        
        // Use GlobalScope for operations that outlive the call handler
        // Or use a dedicated scope with proper lifecycle management
        GlobalScope.launch(Dispatchers.Default) {
            try {
                val file = java.io.File(imagePath)
                if (!file.exists()) {
                    withContext(Dispatchers.Main) {
                        result.error("FILE_NOT_FOUND", "Image not found", null)
                    }
                    return@launch
                }
                
                // Simulate image processing
                val bitmap = android.graphics.BitmapFactory.decodeFile(imagePath)
                val processed = compressBitmap(bitmap, quality)
                
                val outputPath = saveProcessedImage(processed)
                
                // Return on main thread
                withContext(Dispatchers.Main) {
                    result.success(mapOf(
                        "outputPath" to outputPath,
                        "width" to processed.width,
                        "height" to processed.height,
                        "size" to java.io.File(outputPath).length()
                    ))
                }
            } catch (e: Exception) {
                withContext(Dispatchers.Main) {
                    result.error("PROCESSING_ERROR", e.message, null)
                }
            }
        }
    }
    
    private fun compressBitmap(bitmap: android.graphics.Bitmap, quality: Int): android.graphics.Bitmap {
        // Actual implementation would resize/compress
        return bitmap
    }
    
    private fun saveProcessedImage(bitmap: android.graphics.Bitmap): String {
        val file = java.io.File(context.cacheDir, "processed_${System.currentTimeMillis()}.jpg")
        file.outputStream().use { out ->
            bitmap.compress(android.graphics.Bitmap.CompressFormat.JPEG, 90, out)
        }
        return file.absolutePath
    }
    
    fun dispose() {
        mainScope.cancel()
    }
    
    data class User(
        val id: String,
        val name: String,
        val email: String
    ) {
        fun toMap(): Map<String, Any> = mapOf(
            "id" to id,
            "name" to name,
            "email" to email
        )
    }
}
```

**Explanation:**

- **CoroutineScope**: Platform channels run on the main (UI) thread by default. Use `Dispatchers.IO` for network/database operations and `Dispatchers.Default` for CPU-intensive work like image processing.
- **withContext**: Switches coroutine context. Must switch back to `Dispatchers.Main` (or use `result` before suspension) when calling `result.success/error` because platform channel results must be sent on the main thread.
- **GlobalScope**: Used for operations that must complete even if the user navigates away. However, proper lifecycle management using `MainScope` or `viewModelScope` is preferred to avoid memory leaks.
- **Error Handling**: Always wrap async operations in try-catch and return errors via `result.error()` rather than throwing exceptions, which would crash the app.

### **Event Channel with Flow**

```kotlin
// android/app/src/main/kotlin/com/example/app/channels/SensorEventChannel.kt

package com.example.app.channels

import android.content.Context
import android.hardware.Sensor
import android.hardware.SensorEvent
import android.hardware.SensorEventListener
import android.hardware.SensorManager
import io.flutter.plugin.common.EventChannel
import kotlinx.coroutines.*
import kotlinx.coroutines.flow.*

class SensorEventChannel(
    context: Context,
    private val coroutineScope: CoroutineScope
) : EventChannel.StreamHandler {
    
    private val sensorManager = context.getSystemService(Context.SENSOR_SERVICE) as SensorManager
    private var eventSink: EventChannel.EventSink? = null
    
    // Job for collecting flow
    private var collectionJob: Job? = null
    
    // Backing Flow for sensor data
    private val sensorFlow = callbackFlow {
        val listener = object : SensorEventListener {
            override fun onSensorChanged(event: SensorEvent) {
                // Offer data to flow, drop if buffer full (CONFLATED)
                trySend(mapOf(
                    "x" to event.values[0],
                    "y" to event.values[1],
                    "z" to event.values[2],
                    "timestamp" to event.timestamp
                ))
            }
            
            override fun onAccuracyChanged(sensor: Sensor?, accuracy: Int) {
                // Optional: emit accuracy changes
            }
        }
        
        val accelerometer = sensorManager.getDefaultSensor(Sensor.TYPE_ACCELEROMETER)
        sensorManager.registerListener(
            listener, 
            accelerometer, 
            SensorManager.SENSOR_DELAY_GAME  // 50Hz sampling
        )
        
        // Cleanup when flow collector cancelled
        awaitClose {
            sensorManager.unregisterListener(listener)
        }
    }.buffer(Channel.CONFLATED)  // Keep only latest value if consumer slow
    
    override fun onListen(arguments: Any?, events: EventChannel.EventSink?) {
        eventSink = events
        
        // Start collecting flow and sending to Flutter
        collectionJob = coroutineScope.launch {
            sensorFlow
                .flowOn(Dispatchers.Default)
                .collect { data ->
                    events?.success(data)
                }
        }
        
        // Handle errors
        collectionJob?.invokeOnCompletion { throwable ->
            throwable?.let {
                events?.error("SENSOR_ERROR", it.message, null)
            }
        }
    }
    
    override fun onCancel(arguments: Any?) {
        collectionJob?.cancel()
        collectionJob = null
        eventSink = null
    }
}
```

**Explanation:**

- **callbackFlow**: Kotlin Flow builder that converts callback-based APIs (like Android SensorManager) into cold streams.
- **awaitClose**: Ensures sensor listener is unregistered when Flutter stops listening (stream cancelled), preventing battery drain and memory leaks.
- **Channel.CONFLATED**: If Flutter can't process sensor events fast enough (UI jank), drop intermediate values and keep only the latest. Critical for high-frequency data like accelerometers (50-100Hz).
- **flowOn(Dispatchers.Default)**: Sensor callbacks run on a background thread, but we explicitly ensure Flow operations don't block the main thread.

---

## **24.3 Android Manifest Configuration**

Proper manifest configuration is essential for permissions, app links, and hardware features.

### **AndroidManifest.xml Configuration**

```xml
<!-- android/app/src/main/AndroidManifest.xml -->

<manifest xmlns:android="http://schemas.android.com/apk/res/android"
    package="com.example.app">
    
    <!-- Permissions -->
    <uses-permission android:name="android.permission.INTERNET" />
    <uses-permission android:name="android.permission.ACCESS_NETWORK_STATE" />
    
    <!-- Storage permissions based on API level -->
    <uses-permission android:name="android.permission.READ_EXTERNAL_STORAGE" 
        android:maxSdkVersion="32" />
    <uses-permission android:name="android.permission.WRITE_EXTERNAL_STORAGE" 
        android:maxSdkVersion="29" />
    
    <!-- Android 13+ specific permissions -->
    <uses-permission android:name="android.permission.READ_MEDIA_IMAGES" />
    <uses-permission android:name="android.permission.READ_MEDIA_VIDEO" />
    
    <!-- Camera and Hardware -->
    <uses-permission android:name="android.permission.CAMERA" />
    <uses-feature android:name="android.hardware.camera" 
        android:required="false" />
    
    <!-- Location -->
    <uses-permission android:name="android.permission.ACCESS_FINE_LOCATION" />
    <uses-permission android:name="android.permission.ACCESS_COARSE_LOCATION" />
    <uses-permission android:name="android.permission.ACCESS_BACKGROUND_LOCATION" />
    
    <!-- Biometric -->
    <uses-permission android:name="android.permission.USE_BIOMETRIC" />
    
    <!-- Notifications -->
    <uses-permission android:name="android.permission.POST_NOTIFICATIONS" />
    <uses-permission android:name="android.permission.FOREGROUND_SERVICE" />
    
    <application
        android:label="My Flutter App"
        android:icon="@mipmap/ic_launcher"
        android:roundIcon="@mipmap/ic_launcher_round"
        android:allowBackup="false"
        android:fullBackupContent="false"
        android:usesCleartextTraffic="false"
        android:networkSecurityConfig="@xml/network_security_config"
        android:requestLegacyExternalStorage="false">
        
        <!-- Main Activity -->
        <activity
            android:name=".MainActivity"
            android:launchMode="singleTask"
            android:theme="@style/LaunchTheme"
            android:configChanges="orientation|keyboardHidden|keyboard|screenSize|smallestScreenSize|locale|layoutDirection|fontScale|screenLayout|density|uiMode"
            android:hardwareAccelerated="true"
            android:windowSoftInputMode="adjustResize"
            android:exported="true">
            
            <!-- Launcher Intent -->
            <intent-filter>
                <action android:name="android.intent.action.MAIN" />
                <category android:name="android.intent.category.LAUNCHER" />
            </intent-filter>
            
            <!-- Deep Links -->
            <intent-filter>
                <action android:name="android.intent.action.VIEW" />
                <category android:name="android.intent.category.DEFAULT" />
                <category android:name="android.intent.category.BROWSABLE" />
                <data android:scheme="myapp" 
                      android:host="open" />
            </intent-filter>
            
            <!-- App Links (HTTPS) -->
            <intent-filter android:autoVerify="true">
                <action android:name="android.intent.action.VIEW" />
                <category android:name="android.intent.category.DEFAULT" />
                <category android:name="android.intent.category.BROWSABLE" />
                <data android:scheme="https" 
                      android:host="example.com" 
                      android:pathPrefix="/app" />
            </intent-filter>
            
            <!-- Share Intent Receiver -->
            <intent-filter>
                <action android:name="android.intent.action.SEND" />
                <category android:name="android.intent.category.DEFAULT" />
                <data android:mimeType="text/plain" />
                <data android:mimeType="image/*" />
            </intent-filter>
        </activity>
        
        <!-- Flutter Gallery Activity (example of multiple activities) -->
        <activity
            android:name=".GalleryActivity"
            android:theme="@style/NormalTheme"
            android:configChanges="orientation|keyboardHidden|keyboard|screenSize"
            android:hardwareAccelerated="true" />
        
        <!-- Foreground Service for background tasks -->
        <service
            android:name=".services.DataSyncService"
            android:enabled="true"
            android:exported="false"
            android:foregroundServiceType="dataSync" />
        
        <!-- WorkManager initialization -->
        <provider
            android:name="androidx.startup.InitializationProvider"
            android:authorities="${applicationId}.androidx-startup"
            android:exported="false"
            tools:node="merge">
            <meta-data
                android:name="androidx.work.WorkManagerInitializer"
                android:value="androidx.startup" />
        </provider>
        
        <!-- FileProvider for sharing files -->
        <provider
            android:name="androidx.core.content.FileProvider"
            android:authorities="${applicationId}.fileprovider"
            android:exported="false"
            android:grantUriPermissions="true">
            <meta-data
                android:name="android.support.FILE_PROVIDER_PATHS"
                android:resource="@xml/file_paths" />
        </provider>
        
    </application>
</manifest>
```

**Explanation:**

- **maxSdkVersion**: On permissions like `READ_EXTERNAL_STORAGE`, specifying `maxSdkVersion` ensures the permission is only requested on older Android versions. On Android 13+, use granular media permissions (`READ_MEDIA_IMAGES`) instead.
- **android:required="false"**: For hardware features like camera, this allows installation on devices without the hardware. Check availability at runtime using `packageManager.hasSystemFeature()`.
- **android:allowBackup="false"**: Prevents Android from backing up app data to Google Drive. Critical for security if you store sensitive tokens in local databases (though you shouldn't).
- **usesCleartextTraffic="false"**: Enforces HTTPS for all network traffic. Disable only for development with local servers.
- **networkSecurityConfig**: References an XML file for certificate pinning and domain configuration (see Chapter 20).
- **intent-filter**: Multiple filters allow the app to handle different actions: MAIN/LAUNCHER for app start, VIEW for deep links, SEND for receiving shared content.

### **File Provider Configuration**

```xml
<!-- android/app/src/main/res/xml/file_paths.xml -->
<?xml version="1.0" encoding="utf-8"?>
<paths xmlns:android="http://schemas.android.com/apk/res/android">
    <!-- Internal app files -->
    <files-path name="internal_files" path="." />
    
    <!-- Cache directory -->
    <cache-path name="cache_files" path="." />
    
    <!-- External storage (app-specific) -->
    <external-files-path name="external_files" path="Pictures" />
    
    <!-- Shareable directory -->
    <external-path name="shared" path="SharedApp/" />
</paths>
```

---

## **24.4 App Links & Intent Handling**

Handling incoming intents and deep links requires coordination between Android manifest and Flutter code.

### **Deep Link Handler**

```kotlin
// android/app/src/main/kotlin/com/example/app/DeepLinkHandler.kt

package com.example.app

import android.content.Intent
import android.net.Uri
import android.os.Bundle
import io.flutter.embedding.android.FlutterActivity
import io.flutter.plugin.common.MethodChannel

class DeepLinkActivity : FlutterActivity() {
    
    private var initialRoute: String? = null
    private var linkData: Map<String, Any>? = null
    
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        handleIntent(intent)
    }
    
    override fun onNewIntent(intent: Intent) {
        super.onNewIntent(intent)
        // Handle links when activity is already running (singleTask)
        handleIntent(intent)
        
        // Send to Flutter via platform channel
        sendLinkToFlutter(intent)
    }
    
    private fun handleIntent(intent: Intent) {
        when (intent.action) {
            Intent.ACTION_VIEW -> {
                val uri = intent.data
                uri?.let {
                    parseDeepLink(it)
                }
            }
            Intent.ACTION_SEND -> {
                handleShareIntent(intent)
            }
        }
    }
    
    private fun parseDeepLink(uri: Uri) {
        // Parse myapp://open/product/123
        // or https://example.com/app/profile
        
        val pathSegments = uri.pathSegments
        val params = uri.queryParameterNames.associateWith { 
            uri.getQueryParameter(it) 
        }
        
        initialRoute = when (uri.scheme) {
            "myapp" -> {
                // Custom scheme
                "/${pathSegments.joinToString("/")}"
            }
            "https" -> {
                // App link
                when (uri.host) {
                    "example.com" -> {
                        if (pathSegments.firstOrNull() == "app") {
                            "/${pathSegments.drop(1).joinToString("/")}"
                        } else {
                            "/home"
                        }
                    }
                    else -> "/home"
                }
            }
            else -> "/home"
        }
        
        linkData = mapOf(
            "route" to initialRoute,
            "params" to params,
            "uri" to uri.toString()
        )
    }
    
    private fun handleShareIntent(intent: Intent) {
        val type = intent.type
        val sharedText = intent.getStringExtra(Intent.EXTRA_TEXT)
        val sharedUri = intent.getParcelableExtra<Uri>(Intent.EXTRA_STREAM)
        
        linkData = mapOf(
            "action" to "share",
            "type" to type,
            "text" to sharedText,
            "uri" to sharedUri?.toString()
        )
        
        initialRoute = "/share-receiver"
    }
    
    private fun sendLinkToFlutter(intent: Intent) {
        // If engine is already running, send event
        flutterEngine?.let { engine ->
            MethodChannel(engine.dartExecutor.binaryMessenger, "deeplink_channel")
                .invokeMethod("onDeepLink", linkData)
        }
    }
    
    // Called by Flutter to get initial link that started the app
    private fun setupMethodChannel() {
        MethodChannel(flutterEngine!!.dartExecutor.binaryMessenger, "deeplink_channel")
            .setMethodCallHandler { call, result ->
                when (call.method) {
                    "getInitialLink" -> {
                        result.success(linkData)
                    }
                    "clearInitialLink" -> {
                        linkData = null
                        result.success(true)
                    }
                    else -> result.notImplemented()
                }
            }
    }
    
    override fun configureFlutterEngine(flutterEngine: FlutterEngine) {
        super.configureFlutterEngine(flutterEngine)
        setupMethodChannel()
    }
}
```

**Explanation:**

- **onNewIntent**: When `launchMode` is `singleTask`, a new intent to an existing activity calls `onNewIntent` instead of creating a new instance. Must handle both `onCreate` (cold start) and `onNewIntent` (warm start).
- **Intent.ACTION_VIEW**: Standard action for deep links (URL schemes and app links).
- **Intent.ACTION_SEND**: Action when user shares content to your app from another app (Share Sheet).
- **Parcelable**: `Intent.EXTRA_STREAM` returns a `Uri` that implements `Parcelable`. Must cast properly and handle nulls.
- **MethodChannel**: Two-way communication: Flutter calls `getInitialLink` to retrieve data from cold start, native calls `onDeepLink` to send data when app is already running.

### **Dart Side Deep Link Handling**

```dart
// lib/services/deep_link_service.dart

import 'package:flutter/services.dart';
import 'package:uni_links/uni_links.dart';  // Or use platform channels directly

class DeepLinkService {
  static const MethodChannel _channel = MethodChannel('deeplink_channel');
  
  static Future<void> initialize() async {
    // Get initial link (app was started by link)
    try {
      final initialLink = await _channel.invokeMethod('getInitialLink');
      if (initialLink != null) {
        _handleLink(initialLink);
      }
    } catch (e) {
      print('Error getting initial link: $e');
    }
    
    // Listen for links when app is running
    _channel.setMethodCallHandler((call) async {
      if (call.method == 'onDeepLink') {
        _handleLink(call.arguments);
      }
    });
  }
  
  static void _handleLink(dynamic linkData) {
    final route = linkData['route'] as String?;
    final params = linkData['params'] as Map<dynamic, dynamic>?;
    
    if (route != null) {
      // Navigate using your router
      // router.push(route, arguments: params);
    }
  }
}
```

---

## **24.5 Background Services & WorkManager**

Android restricts background execution to conserve battery. Use WorkManager for deferrable tasks and foreground services for immediate user-visible tasks.

### **Foreground Service**

```kotlin
// android/app/src/main/kotlin/com/example/app/services/DataSyncService.kt

package com.example.app.services

import android.app.Notification
import android.app.NotificationChannel
import android.app.NotificationManager
import android.app.PendingIntent
import android.app.Service
import android.content.Intent
import android.os.Build
import android.os.IBinder
import androidx.core.app.NotificationCompat
import com.example.app.MainActivity
import com.example.app.R
import kotlinx.coroutines.*

class DataSyncService : Service() {
    
    private val serviceScope = CoroutineScope(SupervisorJob() + Dispatchers.IO)
    private var isRunning = false
    
    companion object {
        const val CHANNEL_ID = "data_sync_channel"
        const val NOTIFICATION_ID = 1
        const val ACTION_STOP = "STOP_SERVICE"
    }
    
    override fun onCreate() {
        super.onCreate()
        createNotificationChannel()
    }
    
    override fun onStartCommand(intent: Intent?, flags: Int, startId: Int): Int {
        if (intent?.action == ACTION_STOP) {
            stopSelf()
            return START_NOT_STICKY
        }
        
        if (!isRunning) {
            isRunning = true
            startForeground(NOTIFICATION_ID, buildNotification("Syncing data..."))
            
            serviceScope.launch {
                performSync()
            }
        }
        
        return START_STICKY  // Restart if killed by system
    }
    
    private suspend fun performSync() {
        try {
            // Update notification progress
            updateNotification("Uploading files...", 0)
            
            val files = getPendingFiles()
            files.forEachIndexed { index, file ->
                uploadFile(file)
                updateNotification("Uploading...", (index + 1) * 100 / files.size)
            }
            
            updateNotification("Sync complete", 100)
            delay(2000) // Show completion briefly
            stopSelf()
        } catch (e: Exception) {
            updateNotification("Sync failed: ${e.message}", 0)
            delay(5000)
            stopSelf()
        }
    }
    
    private fun getPendingFiles(): List<java.io.File> {
        // Return list of files to sync
        return emptyList()
    }
    
    private suspend fun uploadFile(file: java.io.File) {
        // Simulate upload
        delay(1000)
    }
    
    private fun buildNotification(content: String, progress: Int = 0): Notification {
        val stopIntent = Intent(this, DataSyncService::class.java).apply {
            action = ACTION_STOP
        }
        val stopPendingIntent = PendingIntent.getService(
            this, 0, stopIntent, PendingIntent.FLAG_IMMUTABLE
        )
        
        return NotificationCompat.Builder(this, CHANNEL_ID)
            .setContentTitle("Data Synchronization")
            .setContentText(content)
            .setSmallIcon(R.drawable.ic_sync)  // Add icon to res/drawable
            .setProgress(100, progress, progress == 0)
            .setOngoing(true)
            .addAction(R.drawable.ic_stop, "Stop", stopPendingIntent)
            .build()
    }
    
    private fun updateNotification(content: String, progress: Int) {
        val notification = buildNotification(content, progress)
        val notificationManager = getSystemService(NotificationManager::class.java)
        notificationManager.notify(NOTIFICATION_ID, notification)
    }
    
    private fun createNotificationChannel() {
        if (Build.VERSION.SDK_INT >= Build.VERSION_CODES.O) {
            val channel = NotificationChannel(
                CHANNEL_ID,
                "Data Sync",
                NotificationManager.IMPORTANCE_LOW
            ).apply {
                description = "Shows progress of data synchronization"
            }
            
            val notificationManager = getSystemService(NotificationManager::class.java)
            notificationManager.createNotificationChannel(channel)
        }
    }
    
    override fun onBind(intent: Intent?): IBinder? = null
    
    override fun onDestroy() {
        serviceScope.cancel()
        isRunning = false
        super.onDestroy()
    }
}
```

**Explanation:**

- **Foreground Service**: Must call `startForeground()` within 5 seconds of `onStartCommand()`, passing a notification. This shows a persistent notification to the user indicating the app is doing work.
- **START_STICKY**: If the system kills the service due to memory pressure, it will restart with a null intent. Use this for critical operations.
- **NotificationChannel**: Required on Android 8.0+ (API 26). `IMPORTANCE_LOW` prevents sound/vibration but shows the notification.
- **PendingIntent.FLAG_IMMUTABLE**: Required on Android 12+ (API 31) for security. Prevents mutability of PendingIntents.
- **Service Scope**: Uses `SupervisorJob` so if one child coroutine fails, others continue. Cancelled in `onDestroy()` to prevent leaks.

### **WorkManager for Background Tasks**

```kotlin
// android/app/src/main/kotlin/com/example/app/workers/SyncWorker.kt

package com.example.app.workers

import android.content.Context
import androidx.work.CoroutineWorker
import androidx.work.WorkerParameters
import androidx.work.Data
import androidx.work.Constraints
import androidx.work.NetworkType
import androidx.work.PeriodicWorkRequestBuilder
import androidx.work.WorkManager
import java.util.concurrent.TimeUnit

class SyncWorker(
    context: Context,
    params: WorkerParameters
) : CoroutineWorker(context, params) {
    
    override suspend fun doWork(): Result {
        return try {
            // Get input data
            val userId = inputData.getString("userId") ?: return Result.failure()
            
            // Perform background sync
            val repository = SyncRepository(applicationContext)
            repository.syncData(userId)
            
            // Return output data
            val outputData = Data.Builder()
                .putInt("syncedCount", repository.lastSyncCount)
                .build()
            
            Result.success(outputData)
        } catch (e: Exception) {
            // Retry with exponential backoff if network error
            if (isNetworkError(e)) {
                Result.retry()
            } else {
                Result.failure()
            }
        }
    }
    
    private fun isNetworkError(e: Exception): Boolean {
        return e is java.net.UnknownHostException || 
               e is java.net.SocketTimeoutException
    }
    
    companion object {
        fun schedulePeriodicSync(context: Context, userId: String) {
            val constraints = Constraints.Builder()
                .setRequiredNetworkType(NetworkType.CONNECTED)
                .setRequiresBatteryNotLow(true)
                .build()
            
            val syncWork = PeriodicWorkRequestBuilder<SyncWorker>(
                15, TimeUnit.MINUTES  // Minimum interval is 15 minutes
            ).setConstraints(constraints)
             .setInputData(Data.Builder().putString("userId", userId).build())
             .build()
            
            WorkManager.getInstance(context).enqueueUniquePeriodicWork(
                "sync_work_$userId",
                androidx.work.ExistingPeriodicWorkPolicy.KEEP,
                syncWork
            )
        }
        
        fun cancelSync(context: Context, userId: String) {
            WorkManager.getInstance(context)
                .cancelUniqueWork("sync_work_$userId")
        }
    }
}

// Simple repository for example
class SyncRepository(private val context: Context) {
    var lastSyncCount: Int = 0
    
    suspend fun syncData(userId: String) {
        // Implementation
        lastSyncCount = 10
    }
}
```

**Explanation:**

- **CoroutineWorker**: Modern WorkManager implementation using Kotlin coroutines instead of `ListenableWorker`. Supports suspension and structured concurrency.
- **Constraints**: Define conditions for execution: network available, battery not low, device charging, storage not low, etc.
- **PeriodicWorkRequest**: Minimum repeat interval is 15 minutes (Android system limitation). Use `ExistingPeriodicWorkPolicy.KEEP` to avoid duplicating work if called multiple times.
- **Result Types**: `success()` - work completed, `failure()` - permanent error (don't retry), `retry()` - transient error (exponential backoff retry).
- **Input/Output Data**: Pass primitive data (max 10KB) between WorkManager and Worker. For larger data, save to disk and pass URI.

### **Starting Service from Flutter**

```kotlin
// Method to start service from Flutter
private fun startSyncService() {
    val serviceIntent = Intent(this, DataSyncService::class.java)
    
    if (Build.VERSION.SDK_INT >= Build.VERSION_CODES.O) {
        // Android 8+ requires startForegroundService
        startForegroundService(serviceIntent)
    } else {
        startService(serviceIntent)
    }
}

// Method to schedule work
private fun scheduleBackgroundWork(call: MethodCall, result: MethodChannel.Result) {
    val userId = call.argument<String>("userId") ?: run {
        result.error("INVALID_ID", "User ID required", null)
        return
    }
    
    SyncWorker.schedulePeriodicSync(this, userId)
    result.success(true)
}
```

---

## **Chapter Summary**

In this chapter, we covered deep Android integration patterns for Flutter:

### **Key Takeaways:**

1. **Activity Embedding**: Use `FlutterActivity` for full-screen Flutter, `FlutterFragment` for hybrid UIs. Cache `FlutterEngine` in `Application.onCreate()` to reduce cold start time by 60-70%.

2. **Async Platform Channels**: Use Kotlin Coroutines with `Dispatchers.IO` for network/DB operations. Always call `result.success/error` on main thread. Use `GlobalScope` cautiously; prefer lifecycle-aware scopes.

3. **Event Streams**: Use `callbackFlow` to convert Android callback APIs (sensors, location) to Flows. Apply backpressure strategies (`CONFLATED`) for high-frequency data like accelerometers.

4. **Manifest Configuration**: Use granular permissions for Android 13+ (`READ_MEDIA_IMAGES` vs `READ_EXTERNAL_STORAGE`). Disable `allowBackup` for security-sensitive apps. Configure `FileProvider` for sharing files with other apps.

5. **Deep Links**: Handle both `onCreate` (cold start) and `onNewIntent` (warm start with `singleTask` launchMode). Parse URI components to extract route and parameters.

6. **Background Execution**: Use **WorkManager** for deferrable, guaranteed execution (sync, cleanup) with constraints (network, battery). Use **Foreground Services** only for user-visible tasks (music playback, active navigation) that require immediate execution.

### **Performance & Battery:**

- WorkManager respects Doze mode and App Standby buckets
- Foreground services show persistent notification and consume more battery
- Avoid wakelocks; use WorkManager's expedited work (Android 12+) for urgent tasks

### **Security:**

- FileProvider prevents File URI exposure (FileUriExposedException)
- Immutable PendingIntents prevent intent hijacking on Android 12+
- Clear text traffic disabled enforces HTTPS

### **Next Steps:**

The next chapter (Chapter 25) will cover **Native iOS Integration**, including:
- Swift/Objective-C interop with Flutter
- UIViewController embedding
- iOS-specific plugins and Info.plist configuration
- Universal Links and URL schemes
- Background processing and Background Fetch

---

**End of Chapter 24**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='23. platform_channels.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='25. native_ios_integration.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
